In [ ]:
pip install datasets

In [ ]:
pip install pandas tensorflow transformers

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [ ]:
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("tushkulange/query_builder_text_to_sql")

# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

# Save the DataFrame to a CSV file
df.to_csv("query_data.csv", index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import GPT2Tokenizer

# Load the dataset
df = pd.read_csv("query_data.csv")

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define the maximum sequence length
max_seq_length = 512

# Tokenize the query builder text and SQL queries
tokenized_texts = []
tokenized_queries = []

for idx, row in df.iterrows():
    # Tokenize query builder text
    tokenized_text = tokenizer.encode(row['Input'], return_tensors='tf', max_length=max_seq_length, truncation=True)
    tokenized_texts.append(tokenized_text[0])

    # Tokenize SQL query
    tokenized_query = tokenizer.encode(row['Query'], return_tensors='tf', max_length=max_seq_length, truncation=True)
    tokenized_queries.append(tokenized_query[0])

# Pad or truncate sequences to ensure they have the same length
tokenized_texts = tf.keras.preprocessing.sequence.pad_sequences(tokenized_texts, maxlen=max_seq_length, padding='post', truncating='post')
tokenized_queries = tf.keras.preprocessing.sequence.pad_sequences(tokenized_queries, maxlen=max_seq_length, padding='post', truncating='post')

# Convert tokenized lists to TensorFlow tensors
input_ids = tf.convert_to_tensor(tokenized_texts)
labels = tf.convert_to_tensor(tokenized_queries)

# Print example of tokenized input and labels
print("Tokenized input:", tokenizer.decode(input_ids[0]))
print("Tokenized label:", tokenizer.decode(labels[0]))


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Tokenized input: give me the last month body temperature for this patient id?!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Tokenized label: SELECT * FROM biometrics_body_temperature WHERE PatientUserId = 'patient_uuid_here' AND RecordDate >= NOW() - INTERVAL 1 MONTH;!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)


# Compile the model with appropriate optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
def custom_loss(y_true, y_pred):
    # Compute the loss using mean squared error (MSE) as an example
    loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return loss
    model.compile(optimizer=optimizer, loss=custom_loss)
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer,custom_loss)



In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [ ]:
model.fit(input_ids, labels, epochs=1,batch_size=4)

In [ ]:
validation_dataset = tf.data.Dataset.from_tensor_slices((input_ids, labels)).batch(4)

# Evaluate the model on the validation dataset
evaluation_metrics = model.evaluate(validation_dataset)


114/114 [==============================] - 2221s 19s/step - loss: 10.8249


In [ ]:
model.save("fine_tuned_model")

In [ ]:
import shutil
shutil.make_archive("fine_tuned_model", "zip", "fine_tuned_model")


'/content/fine_tuned_model.zip'

In [ ]:
from google.colab import files



In [ ]:
files.download("fine_tuned_model.zip")